# Borrowing Constraint

Last update: Nov 12th, 2020

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
%matplotlib inline

In [4]:
# Parameterization
beta = 0.96
R = 1.01
gamma = 2.0

In [5]:
# Assume the log of income is an AR(1) process, discretized by Tauchen method
rho = 0.8
sigma = 0.1
m = 3
N = 15

def Tauchen(rho, sigma, m, N):
    y_N = m * np.sqrt(sigma ** 2 / (1 - rho ** 2))
    y_grid = np.linspace(-y_N, y_N, N)
    d = 2 * y_N / (N - 1)
    F = stats.norm().cdf
    
    Pi = np.zeros((N, N))
    for i in range(N):
        for j in range(N):
            if (j == 0):
                lb = 0.0
            else:
                lb = F((y_grid[j] - (d / 2) - rho * y_grid[i]) / sigma)
                
            if (j == N - 1):
                rb = 1.0
            else:
                rb = F((y_grid[j] + (d / 2) - rho * y_grid[i]) / sigma)
    
            Pi[i, j] = rb - lb

    return y_grid, Pi

y_grid, Pi = Tauchen(rho, sigma, m, N)

In [7]:
# Grid for savings

n_k = 1000
k_min = 0.0
k_max = 20.0
k_grid = np.linspace(k_min, k_max, n_k)

In [8]:
def u_vec(c, gamma):
    if (gamma == 1.0):
        res = np.log(c)
    else:
        res = (c ** (1 - gamma)) / (1 - gamma)
    
    res[np.isnan(res)] = -1e10
    
    return res